# 第8章: 機械学習

本章では，Bo Pang氏とLillian Lee氏が公開しているMovie Review Dataのsentence polarity dataset v1.0を用い，文を肯定的（ポジティブ）もしくは否定的（ネガティブ）に分類するタスク（極性分析）に取り組む．

In [13]:
!head  rt-polaritydata/rt-polaritydata/rt-polarity.neg

simplistic , silly and tedious . 
it's so laddish and juvenile , only teenage boys could possibly find it funny . 
exploitative and largely devoid of the depth or sophistication that would make watching such a graphic treatment of the crimes bearable . 
[garbus] discards the potential for pathological study , exhuming instead , the skewed melodrama of the circumstantial situation . 
a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification . 
the story is also as unoriginal as they come , already having been recycled more times than i'd care to count . 
about the only thing to give the movie points for is bravado -- to take an entirely stale concept and push it through the audience's meat grinder one more time . 
not so much farcical as sour . 
unfortunately the story and the actors are served with a hack script . 
all the more disquieting for its relatively gore-free allusions to the serial murders , but it falls down in its attempts to humaniz

## 70. データの入手・整形
文に関する極性分析の正解データを用い，以下の要領で正解データ（sentiment.txt）を作成せよ．

1. rt-polarity.posの各行の先頭に"+1 "という文字列を追加する（極性ラベル"+1"とスペースに続けて肯定的な文の内容が続く）
2. rt-polarity.negの各行の先頭に"-1 "という文字列を追加する（極性ラベル"-1"とスペースに続けて否定的な文の内容が続く）
3. 上述1と2の内容を結合（concatenate）し，行をランダムに並び替える

sentiment.txtを作成したら，正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ．

In [90]:
input1 ="rt-polarity.pos"
input2="rt-polarity.neg"
output="sentiment.txt"

words = []
with open(input1, 'r') as pos:
    words.extend(['+1 {}'.format(line.strip()) for line in pos])
with open(input2, 'r') as neg:
    words.extend(['+1 {}'.format(line.strip()) for line in neg])
import random
random.shuffle(words)

with open(output, 'w') as file_out:
    print(*words, sep='\n', file=file_out)

In [98]:
input1 ="rt-polarity.neg" #5332
input2="rt-polarity.pos" #5331
#input1 = 'rt-polaritydata/rt-polaritydata/rt-polarity.pos'
#input2 = 'rt-polaritydata/rt-polaritydata/rt-polarity.neg'
output="sentiment.txt"
f = open(output,"w")
words=[]
for line in open(input1):
    words.append("-1 "+ line)
    #writelines("-1 "+line)
print("neg>",len(words))
for line in open(input2):
    words.append("+1 "+ line)
    #writelines("+1 "+line)
    
import random
random.shuffle(words)
#出力
for line in words:
    print(line,end="",file=f)

neg> 5331


In [14]:
import codecs
import random

fname_pos = 'rt-polaritydata/rt-polaritydata/rt-polarity.pos'
fname_neg = 'rt-polaritydata/rt-polaritydata/rt-polarity.neg'
fname_smt = 'sentiment.txt'
fencoding = 'cp1252'        # Windows-1252らしい

result = []

# ポジティブデータの読み込み
with codecs.open(fname_pos, 'r', fencoding) as file_pos:
    result.extend(['+1 {}'.format(line.strip()) for line in file_pos])

# ネガティブデータの読み込み
with codecs.open(fname_neg, 'r', fencoding) as file_neg:
    result.extend(['-1 {}'.format(line.strip()) for line in file_neg])

# シャッフル
random.shuffle(result)

# 書き出し
with codecs.open(fname_smt, 'w') as file_out:
    print(*result, sep='\n', file=file_out)

In [15]:
!head -n 10 sentiment.txt

+1 [anderson] uses a hit-or-miss aesthetic that hits often enough to keep the film entertaining even if none of it makes a lick of sense .
-1 you might be shocked to discover that seinfeld's real life is boring .
-1 needless to say , the dramatics that follow are utter hooey .
+1 for the most part , it works beautifully as a movie without sacrificing the integrity of the opera .
+1 grant carries the day with impeccable comic timing , raffish charm and piercing intellect .
+1 you'll gasp appalled and laugh outraged and possibly , watching the spectacle of a promising young lad treading desperately in a nasty sea , shed an errant tear .
-1 it's just weirdness for the sake of weirdness , and where human nature should be ingratiating , it's just grating .
-1 it's provocative stuff , but the speculative effort is hampered by taylor's cartoonish performance and the film's ill-considered notion that hitler's destiny was shaped by the most random of chances .
-1 [a] shapeless blob of desperate

In [16]:
!wc rt-polarity.neg
!wc rt-polarity.pos
!wc sentiment.txt

wc: rt-polarity.neg: open: No such file or directory
wc: rt-polarity.pos: open: No such file or directory
   10662  234730 1260267 sentiment.txt


In [84]:
!cat rt-polarity.pos rt-polarity.neg > sentiment.txt
!wc sentiment.txt

   10661  223947 1238405 sentiment.txt


In [82]:
pos_count,neg_count =0,0
for line in open('sentiment.txt'):
    if line[0:2]=="+1":
        pos_count+=1
    elif line[0:2] =="-1":
        neg_count+=1
    else:
        print(line)
print(pos_count,neg_count)

5310 5299


## 71. ストップワード
英語のストップワードのリスト（ストップリスト）を適当に作成せよ．さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，それ以外は偽を返す関数を実装せよ．さらに，その関数に対するテストを記述せよ．

【stop word】あまりにたくさん検索にかかりすぎるので、検索精度の向上のためには検索対象から除外せざるを得ない語。助詞や助動詞などの機能語(日本語ならば「は」「の」「です」「ます」など、英語ならば「the」「of」「is」など)は、殆んどの場合これに該当する。

In [134]:
from collections import Counter
words=[]
countA = Counter()
for line in open('sentiment.txt'):
    countA += Counter(line.split(" ")[1:])

In [152]:
stop_word=[]
for i,j in countA.most_common(30):
    stop_word.append(i)
print("STOP WORD: ",stop_word)

STOP WORD:  ['.', '\n', 'the', ',', 'a', 'and', 'of', 'to', 'is', 'in', 'that', 'it', 'as', 'but', 'with', 'film', 'for', 'this', 'its', 'an', 'movie', "it's", 'be', 'on', 'you', 'not', 'by', 'about', 'one', 'more']


### (´･ω･｀)< sentiment.txtの頻度を調べてみた. "movie"とか頻出に入ってしまった(；´∀｀)

In [154]:
def include_stopword(word,stop_word):
    if word in stop_word:
        return True
    else:
        return False

In [155]:
include_stopword("Hello",stop_word)

False

In [160]:
include_stopword("not",stop_word)

True

>ストップワード(stopword)  
文書中の ”the”, ”is”, “have”, “take”など、
話題の種類と関連を持たないと考えられる語。
ストップワードをあらかじめ削除してからベクトル化を行うことが多い。

## 72. 素性抽出
極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．

>ステミング(stemming)  
同じような話題を指している語を一つの素性としてみなしたいとき、
派生語なども含めて同一の素性とみなす作業のこと。
例. ”run”, “runs”, “ran” さらに”runner”など。

>ポーターのステマー(Prter's stemmer)  
英語のステミングの手法. 多くの規則がある.  
規則例
- 語尾のedを除去する(walked→walk)
- 語尾のateを除去する(passionate→passion)
- 語尾のationalを除去する(operational→oper)
  
>“operational”と ”operate”は同じ ”oper” として扱えるが、
(hundred→hundr)となってしまったり、
(international→intern)と、意味が変わってしまうこともある。

In [153]:
from nltk.corpus import stopwords
stopword_list = stopwords.words('english')
stopword_list.append(",")
stopword_list.append(".")

def extract_feature(filename="sentiment.txt"):
    target,feature = [],[]
    x=[]
    for line in open(filename):
        words = line.strip("\n").split()
        y = words[0]
        for word in words[1:]:
            word = stemmer.stem(word)
            if not word in stopword_list:
                x.append(word)
            else:
                pass

        target.append(int(y))
        feature.append(x)
        #init
        x = []
    return target,feature

In [154]:
y_train , x_train = extract_feature()

In [140]:
for y,x in zip(y_train,x_train):
    print(y,x)

1 ['[anderson]', 'use', 'hit-or-miss', 'aesthet', 'hit', 'often', 'enough', 'keep', 'film', 'entertain', 'even', 'none', 'make', 'lick', 'sens', '.']
-1 ['might', 'shock', 'discov', "seinfeld'", 'real', 'life', 'bore', '.']
-1 ['needless', 'say', 'dramat', 'follow', 'utter', 'hooey', '.']
1 ['part', 'work', 'beauti', 'movi', 'without', 'sacrif', 'integr', 'opera', '.']
1 ['grant', 'carri', 'day', 'impecc', 'comic', 'time', 'raffish', 'charm', 'pierc', 'intellect', '.']
1 ["you'll", 'gasp', 'appal', 'laugh', 'outrag', 'possibl', 'watch', 'spectacl', 'promis', 'young', 'lad', 'tread', 'desper', 'nasti', 'sea', 'shed', 'errant', 'tear', '.']
-1 ["it'", 'weird', 'sake', 'weird', 'human', 'natur', 'ingrati', "it'", 'grate', '.']
-1 ["it'", 'provoc', 'stuff', 'specul', 'effort', 'hamper', "taylor'", 'cartoonish', 'perform', "film'", 'ill-consid', 'notion', "hitler'", 'destini', 'wa', 'shape', 'random', 'chanc', '.']
-1 ['[a]', 'shapeless', 'blob', 'desper', 'entertain', '.']
1 ['thi', 'one',

In [134]:
!head -n 5 sentiment.txt

+1 [anderson] uses a hit-or-miss aesthetic that hits often enough to keep the film entertaining even if none of it makes a lick of sense .
-1 you might be shocked to discover that seinfeld's real life is boring .
-1 needless to say , the dramatics that follow are utter hooey .
+1 for the most part , it works beautifully as a movie without sacrificing the integrity of the opera .
+1 grant carries the day with impeccable comic timing , raffish charm and piercing intellect .


## 73. 学習
72で抽出した素性を用いて，ロジスティック回帰モデルを学習せよ．

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

In [165]:
from collections import Counter
x_list = []
for line in x_train:
    x_list.append(dict(Counter(line)))

from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer()
X = v.fit_transform(x_list)

In [168]:
from sklearn.linear_model import LogisticRegression

In [171]:
clf = LogisticRegression()
clf.fit(X,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## 74. 予測
73で学習したロジスティック回帰モデルを用い，与えられた文の極性ラベル（正例なら"+1"，負例なら"-1"）と，その予測確率を計算するプログラムを実装せよ．

In [188]:
res = clf.predict(X[0])

In [189]:
res

array([1])

In [186]:
len(res)

10662

In [187]:
!wc sentiment.txt

   10662  234730 1260267 sentiment.txt


In [191]:
X[0]

<1x15892 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [ ]:
v = DictVectorizer()
X = v.fit_transform(x_list)